# 데이터분석 test notebook

In [84]:
# !pip install seaborn
# !pip install tqdm
# !pip install -U scikit-learn
# !pip install emoji
# !pip install pyarrow
# !pip install fastparquet

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from emoji import core

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
firedragonyes = ['같다', '없다', '이렇다', '다르다', '어떻다', '높다', '길다' ,'크다','있다','없다다','빨갛다','붉다','죠다']

In [86]:
milk_t = pd.read_csv('./test_blog_crawling.csv')
df = milk_t.copy()
df

,Unnamed: 0,Title,contents
0,0,밀크T 패드 갤럭시탭 A6 (배터리 교체) : 네이버 블로그,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서, 또 연장함.\n연장해..."
1,1,밀크티추천인 혜택 / 유아학습지 밀크T로 한글.. : 네이버블로그,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 ~!\n작년 12월 말, 급한 마음..."
2,2,"밀크티 학습지 2년 사용 후기, 초등밀크T 친.. : 네이버블로그","밀크T 친구 추천 이벤트!\n밀크티 학습지 남매 2년 사용, 연장 후기!\n*3월9..."
3,3,천재교육 밀크티 초등 가격 추천인 이달의 프로.. : 네이버블로그,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...
4,4,중학교인강 정착지 밀크T 한 가지로 멀티학습 : 네이버 블로그,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요-\n영어...
...,...,...,...
946,946,Error,Error
947,947,Error,Error
948,948,Error,Error
949,949,Error,Error


In [87]:
df = df[df['Title'] != 'Error']
df.reset_index(drop=True, inplace=True)
df = df.drop(['Title', 'Unnamed: 0'], axis=1)
df

,contents
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서, 또 연장함.\n연장해..."
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 ~!\n작년 12월 말, 급한 마음..."
2,"밀크T 친구 추천 이벤트!\n밀크티 학습지 남매 2년 사용, 연장 후기!\n*3월9..."
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요-\n영어...
...,...
606,"♣ 당신이 병드는 이유 / 콜린캠벨, 하워드 제이콥슨\n우리가 건강을 지키는 데 어..."
607,붓다의 생애와 사상\n불타(佛陀)와 불전(佛傳)\n마성/ 팔리문헌연구소장\n설법연구...
608,*단말기 완전자급제 (출처 : 기기 구입-통신사 선택 ‘따로’… 유통구조 격변 예고...
609,최근 수정 시각: 2019-11-20 00:18:19\n분류\n연습생\n아이돌\nK...


In [88]:
test_sentence = df['contents'][609]
test_sentence[:1000]

'최근 수정 시각: 2019-11-20 00:18:19\n분류\n연습생\n아이돌\nK-POP 관련 문서\n일반\nK-POP · 한류 음악산업 통계 · 한국 음원시장\n아티스트 목록 및 형태별 분류\n한국 가수 · 한국 아이돌 (목록 / 역사)\n걸그룹 (목록 / 역사 / 포지션) · 보이그룹 (목록 / 포지션) · 혼성그룹\n유닛 · 남성 솔로 · 여성 솔로 (목록) · 2인조 · 3인조 · 4인조 · 5인조 · 6인조 · 7인조 · 8인조 · 9인조 · 10인조 이상\n그룹이 변경된 아이돌\n매니지먼트 및 프로듀싱\n연예 기획사 (대형 기획사 / 3대: SM, JYP, YG) · 프로듀서 · 매니저 · 연습생 (조언) · 서바이벌 프로그램\n팬 문화\n팬덤 (목록) · 응원도구 · 응원문구 · 굿즈 (그룹별) · 삼촌팬 · 홈마 · 직캠 · 안무영상 · 팬송 · 팬사인회 · 소속사 안티 · MR 제거 · 리액션 비디오 · 커버 · V LIVE · 교차편집\n각종 기록 및 순위\n가온차트 (가온 인증 / 연간 1위) · 한터차트 · 아이차트 · 멜론차트 (주간 1위 / 월간 1위 / 연간 1위) · 초동 순위 (걸그룹 / 보이그룹) · 총판 순위 (걸그룹) · 빌보드 차트 한국 가수 기록 · 유튜브 K-POP 조회수 순위 · 대중음악 시상식 · 한국 대중음악 100대 명반\n주요 음악방송\n쇼! 음악중심 (차트 / 순위) · 뮤직뱅크 (차트 / 순위) · SBS 인기가요 (차트 / 순위)\n엠 카운트다운 (순위) · 쇼챔피언 (순위) · THE SHOW · Simply K-Pop\n논란 및 문제점\n음원 사재기 · 음반 사재기 · 아이돌 7년차 징크스 · 악성 개인팬 · 사생팬\n1. 인턴2. 아이돌 연습생\n2.1. 오디션 및 데뷔 과정\n2.1.1. 연습생/조언2.1.2. 사회적 인지도2.1.3. 기획사 오디션 준비 전2.1.4. 연습생 오디션 응시 방법2.1.5. 연습생 트레이닝 과정2.1.6. 공연, 쇼케이스2.1.7. 학업 문제\n2.2. 데뷔\n2.

In [89]:
test_sentence2 = df['contents'][4]
test_sentence2[:200]

'연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요-\n영어 수학 외에 다른 과목은 학교 수업만으로 진도를 따라가는데 무리가 없어서 한시름 놓고 있어요.\n스마트 기기로 하는 학습으로는 몇 번의 실패 끝에 정착한 거라 앞으로도 쭉 시키려고 해요-\n요즘 초등학교는 저희 때처럼 중간고사나 기말고사를 따로 보지 않고 상시로 평가를 하는데, 점'

In [90]:
# def search_milkT(text):
#     if "밀크티" not in text:
#         if "밀크t" not in text.lower():
#             if "milk" not in text.lower():
#                 if "ㅁㅋㅌ" not in text:
#                     return False

#     else:
#         return True

In [91]:
def search_milkT(text):
    if "밀크티" in text:
        return True

    elif "밀크t" in text.lower():
        return True
            
    elif "milkT" in text.lower():
        return True
    
    elif "ㅁㅋㅌ" in text:
        return True

    # elif "학습" in text:
    #     return True

    else:
        return False

In [92]:
def check_ad(text):
    if '광고' in text:
        return True
    
    elif '추천인' in text:
        return True

    elif '서포터' in text:
        return True

    else:
        return False

In [93]:
df['Check_milkT'] = df['contents'].apply(lambda x : search_milkT(x))
df['check_test'] = df['contents'].apply(lambda x : '학습' in x and '선생' in x)

# df.Check_milkT.value_counts()
# df['check_test'].value_counts()

df = df[df['check_test'] == True]
df = df[df['Check_milkT'] == True]

# df['check_test2'] = df['contents'].apply(lambda x : '선생' in x)
# df= df[df['check_test2'] == True]

df['recommend'] = df['contents'].apply(lambda x : check_ad(x))
df = df.drop(['Check_milkT', 'check_test'], axis=1)
df.reset_index(drop=True, inplace=True)

In [94]:
# # dataframe 보기 설정
# pd.set_option('display.max_row', 20)

In [95]:
df.contents = df.contents.apply(lambda x : re.sub('[-=+#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\n★▲;]','', x))
df.contents = df.contents.apply(lambda x : re.sub('\W. ','',x))
df.contents = df.contents.apply(lambda x : core.replace_emoji(x, replace=''))
display(df[:10])
print(len(df))

,contents,recommend
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함.연장해서 1년이...",False
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바로...",True
2,"밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요2...",True
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...,True
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요영어 수학...,False
5,우리딸 천재교육 밀크티 2년 넘게 해서 재연장한 상태 입니다.3학년때는 과목도 늘어...,True
6,지난주 설연휴는보내셨나요설연휴부터 코로나 확진자 수가 엄청나네요.3만명이 넘을수도 ...,False
7,안녕하세요요즘에는 아이들이 영어나 수학을배우는 시기가빨라지는같아요.학교에 가기 전부...,False
8,"천재교육 밀크티 가격 12 개월149,000 원24 개월109,000 원1월 3일...",True
9,시골로 이사오면서 심각한 문제가 생겼는데바로 쭈양이 심심하다는 것이다.나는 출근하면...,True


123


In [96]:
contents_list = list(df.contents)
s = contents_list[0]
s

'3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함.연장해서 1년이 지났고 1년이 남은 상태.처음엔 수학 과목만 생각해서 시작을 했는데 코로나로 인해 학교가는 기간이 있어서잘했다 싶다.사회, 과학 관련 내용을수 있다.코딩도 하고, 한자, 중국어도 가능하니..따로 학원필요 없이 1주일에 12회 보며 학습하는 중.밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음.처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다.진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면.일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음. ㅎㅎ그래서 우리남매는 모르는..................계속 몰랐으면 한다3년하고 1개월이 지난같다.여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다. 전원을 켜면 배터리는 다시 0로 꺼짐.학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다.충전이 어려울같아서 학습도 안되고 수업도 안될같다.담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다. 15883366배터리 교체 비용을 물으니 66,000원이라고 한다.더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다.배터리 재고는하나 있다고 하셨다.재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다.하나 남은 재고 예약까지 했다.2주 이내로 방문하면 된다고 함.예약하고 다음날 바로 감. 배터리 교체하고 학습해야 하니까..........오후에 가니 확실히붐볐다.대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다.배터

In [97]:
def point_reduce(sentence):
    while True:
        try:
            if '..' in sentence:
                sentence = sentence.replace('..','.')
            
            else:
                sentence = sentence.replace('.','. ') # 온점 뒤에 space 넣어뒀음. 불필요시 제거.
                if '  ' in sentence:
                    sentence = sentence.replace('  ',' ')
                    # print(sentence)
                    return sentence
                    break
                else:
                    return sentence
        except:
            return sentence
        
        # if '  ' not in sentence:
        #     break

In [98]:
def classify_sentence(sentence):
    sentence_list = sentence.split('. ')
    if ' ' in sentence_list:
        sentence_list.remove(' ')
        return sentence_list

    else:
        return sentence_list

In [99]:
df2 = df.copy()
df2.head()

,contents,recommend
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함.연장해서 1년이...",False
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바로...",True
2,"밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요2...",True
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...,True
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요영어 수학...,False


In [100]:
df['contents'] = df['contents'].apply(lambda x: point_reduce(x))
df['contents_list'] = df['contents'].apply(lambda x: classify_sentence(x))
display(df['contents'][50])
display(df['contents_list'][50])

'아이가 초등학교 2학년 겨울부터시작한 패드학습지패드로 공부한다는게당시엔 생소했지만지나고 보니 좋은 선택이었어요올해 5학년이첫째와7살 예비초등이둘째 아이가천재교육 밀크티로 실제 학습한 후기입니다천재교육 밀크티 가격, 밀크티아이, 초등, 할인 혜택, 추천인, 패드학습재생 수6220000 0032화질 선택 옵션자동천재교육 밀크티 가격, 밀크티아이, 초등, 할인 혜택, 추천인, 패드학습먼저 이번에 7살 예비초등이 된둘째가 영어 공부를 하는 모습이에요작년에 찍은 거라 이땐 6살이었어요태블릿 마이크가 목소리를인식해서 평가를 해줘서 아이도좋은 평가를 받기 위해크고 정확하게말하려고 노력해요패드학습지는 이렇게 보고 듣고 쓰면서학습이 가능해서 아이들이 좋아해요책을 읽고 나서 독후 활동도 중요하다고생각하는데 동화를 보고 나면동화와 관련된 그림을 그리거나문제를 풀어볼있어요한글이나 영어단어를때순서대로 쓰지 않으면 입력이 안돼서자연스럽게쓰는 순서도 배워요초등학생 5학년이 되는 첫째는요즘 5학년 1학기 공부중이에요수학은 6단원5단원 공부중인데이렇게 방학동안 미리 예습을 한번 하고학기가 시작되면 다시 1학기 1단원부터복습하는 방식으로 학습이 진행돼요강의를 보고 마지막에 문제를 풀어서아이가 학습을 제대로 이해했는지부족한 점은 무엇인지 체크할있어요틀린문제는 쌍둥이 문제가 나와서다시 한번 풀어볼있어요담당선생님 수업은 일주일에 한번씩밀크티 학습 패드를 이용한 화상수업이랑전화수업이 격주로 510분정도 진행돼요아이가 학습을 하다가 궁금한 점이 있으면선생님께 쪽지로 질문을 하고답변을 받기도 하고진도도 조절할있어서진도가 빠르거나 느릴때 선생님께 요청하면바로 바로 반영해주세요패드학습이 끝나면 교재로다시 한번 복습하기패드로만 공부해도 충분하지만교재가 있어서꼼꼼하게 공부할있어요교재는 여러종류 교과문제를 반영한빅데이터와 서술형 문제집 와이,밀크티 초등수학기초연산 교재 등으로구성됐어요12개월 약정24개월 약정 할인밀크티아이가격149,000원119,000원밀크티키즈 가격149,000원119,000원밀크티초등14

['아이가 초등학교 2학년 겨울부터시작한 패드학습지패드로 공부한다는게당시엔 생소했지만지나고 보니 좋은 선택이었어요올해 5학년이첫째와7살 예비초등이둘째 아이가천재교육 밀크티로 실제 학습한 후기입니다천재교육 밀크티 가격, 밀크티아이, 초등, 할인 혜택, 추천인, 패드학습재생 수6220000 0032화질 선택 옵션자동천재교육 밀크티 가격, 밀크티아이, 초등, 할인 혜택, 추천인, 패드학습먼저 이번에 7살 예비초등이 된둘째가 영어 공부를 하는 모습이에요작년에 찍은 거라 이땐 6살이었어요태블릿 마이크가 목소리를인식해서 평가를 해줘서 아이도좋은 평가를 받기 위해크고 정확하게말하려고 노력해요패드학습지는 이렇게 보고 듣고 쓰면서학습이 가능해서 아이들이 좋아해요책을 읽고 나서 독후 활동도 중요하다고생각하는데 동화를 보고 나면동화와 관련된 그림을 그리거나문제를 풀어볼있어요한글이나 영어단어를때순서대로 쓰지 않으면 입력이 안돼서자연스럽게쓰는 순서도 배워요초등학생 5학년이 되는 첫째는요즘 5학년 1학기 공부중이에요수학은 6단원5단원 공부중인데이렇게 방학동안 미리 예습을 한번 하고학기가 시작되면 다시 1학기 1단원부터복습하는 방식으로 학습이 진행돼요강의를 보고 마지막에 문제를 풀어서아이가 학습을 제대로 이해했는지부족한 점은 무엇인지 체크할있어요틀린문제는 쌍둥이 문제가 나와서다시 한번 풀어볼있어요담당선생님 수업은 일주일에 한번씩밀크티 학습 패드를 이용한 화상수업이랑전화수업이 격주로 510분정도 진행돼요아이가 학습을 하다가 궁금한 점이 있으면선생님께 쪽지로 질문을 하고답변을 받기도 하고진도도 조절할있어서진도가 빠르거나 느릴때 선생님께 요청하면바로 바로 반영해주세요패드학습이 끝나면 교재로다시 한번 복습하기패드로만 공부해도 충분하지만교재가 있어서꼼꼼하게 공부할있어요교재는 여러종류 교과문제를 반영한빅데이터와 서술형 문제집 와이,밀크티 초등수학기초연산 교재 등으로구성됐어요12개월 약정24개월 약정 할인밀크티아이가격149,000원119,000원밀크티키즈 가격149,000원119,000원밀크티초등1

In [101]:
# vector = CountVectorizer()
# bow = vector.fit_transform()

# print(bow.toarray())
# print(vector.vocabulary_)

In [102]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash
# !pip install konlpy

### mecab install try
## ! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
## ! bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh
## !pip install eunjeon

In [103]:
from konlpy.tag import Kkma, Hannanum, Komoran, Twitter, Okt

In [104]:
Kkma = Kkma()
okt = Okt()
Komoran = Komoran()
hannanum = Hannanum()
twitter = Twitter()
# mecab = Mecab()

c:\ProgramData\Anaconda3\envs\py37\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [105]:
Komoran.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NA': '분석불능범주',
 'NF': '명사추정범주',
 'NNB': '의존 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'NV': '용언추정범주',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

In [106]:
df.head()

,contents,recommend,contents_list
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함. 연장해서 1년...",False,"[3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함, 연장해서 1..."
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바로...",True,"[밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바..."
2,"밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요2...",True,"[밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요..."
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...,True,"[새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠, 저는 올해 중학생..."
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요영어 수학...,False,"[연두는 현재 6학년으로 예비중등입니다, 저학년 때부터 밀크티에 정착했는데요영어 수..."


In [107]:
from nltk.tokenize import word_tokenize

tokens = word_tokenize(s)
tokens

['3학년',
 '때부터',
 '밀크T',
 '수업을',
 '시작했고',
 '2년',
 '약정으로',
 '시작해서',
 ',',
 '연장함.연장해서',
 '1년이',
 '지났고',
 '1년이',
 '남은',
 '상태.처음엔',
 '수학',
 '과목만',
 '생각해서',
 '시작을',
 '했는데',
 '코로나로',
 '인해',
 '학교가는',
 '기간이',
 '있어서잘했다',
 '싶다.사회',
 ',',
 '과학',
 '관련',
 '내용을수',
 '있다.코딩도',
 '하고',
 ',',
 '한자',
 ',',
 '중국어도',
 '가능하니',
 '..',
 '따로',
 '학원필요',
 '없이',
 '1주일에',
 '12회',
 '보며',
 '학습하는',
 '중.밀크T',
 '초등',
 '2020년에',
 '시작할갤럭시탭',
 'A6를',
 '기계로',
 '받았다지금은주는지',
 '모름',
 '밀크아이는',
 '갤럭시탭S',
 '시리즈는',
 '받았다고',
 '들음.처음',
 '커버',
 '벗겨',
 '보는같다패드가',
 '달린',
 '파란',
 '커버가',
 '씌워져있는데',
 '벗기기',
 '힘들어서번도하다가이번에',
 'AS',
 '받을',
 '일이',
 '있어서',
 '벗겨본다.진짜',
 '새것',
 '같음밀크T',
 '학습',
 '화면으로',
 '되어있는데',
 ',',
 '나가기를',
 '누르면.일반',
 '탭으로도',
 '사용이',
 '가능하지만번도',
 '나가기를',
 '누르지',
 '않음',
 '.',
 'ㅎㅎ그래서',
 '우리남매는',
 '모르는',
 '..................',
 '계속',
 '몰랐으면',
 '한다3년하고',
 '1개월이',
 '지난같다.여태사용했는데',
 ',',
 '주말이',
 '지나고',
 '충전하려고',
 '하는데',
 '3시간이',
 '넘게',
 '충전을',
 '해도',
 '1에서이상',
 '올라가지',
 '않는다',
 '.',
 '전원을',
 '켜면',
 '배터리는',
 '다시',
 '0로',
 '꺼짐.학습을

In [108]:
print('Kkma 형태소 분석 : ', Kkma.pos(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.pos(s[:20]))
print('Komoran 형태소 분석 : ', Komoran.pos(s[:20]))
print('Twitter 형태소 분석 : ', twitter.pos(s[:20]))
print('Okt 형태소 분석 : ', okt.pos(s[:20]))
# print('Mecab 형태소 분석 : ', mecab.pos(sentence))

# '3학년', '밀크T'를 묶는 모습에서 Hannaum이 일단은 제일 나아보인다

Kkma 형태소 분석 :  [('3', 'NR'), ('학년', 'NNG'), ('때', 'NNG'), ('부터', 'JX'), ('밀크', 'NNG'), ('T', 'OL'), ('수업', 'NNG'), ('을', 'JKO'), ('시작하', 'VV'), ('었', 'EPT'), ('고', 'ECE')]
Hannaum 형태소 분석 :  [('3학년', 'N'), ('때', 'N'), ('부터', 'J'), ('밀크T', 'N'), ('수업', 'N'), ('을', 'J'), ('시작', 'N'), ('하', 'X'), ('었고', 'E')]
Komoran 형태소 분석 :  [('3', 'SN'), ('학년', 'NNG'), ('때', 'NNG'), ('부터', 'JX'), ('밀크', 'NNP'), ('T', 'SL'), ('수업', 'NNG'), ('을', 'JKO'), ('시작', 'NNG'), ('하', 'XSV'), ('았', 'EP'), ('고', 'EC')]
Twitter 형태소 분석 :  [('3', 'Number'), ('학년', 'Noun'), ('때', 'Noun'), ('부터', 'Josa'), ('밀크', 'Noun'), ('T', 'Alpha'), ('수업', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('했고', 'Verb')]
Okt 형태소 분석 :  [('3', 'Number'), ('학년', 'Noun'), ('때', 'Noun'), ('부터', 'Josa'), ('밀크', 'Noun'), ('T', 'Alpha'), ('수업', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('했고', 'Verb')]


In [109]:
# !pip install textblob

In [110]:
# 키워드 찾기 기능?!
# 밀크T 가 온전히 나오는게 나쁘지 않을듯 하기도 하구...
# 혹은 n-gram추출을 통해서 밀크+T 아이스크림+홈런 같은 식으로 검색을 진행해봐도 괜찮을 듯 하다.

print('Kkma 형태소 분석 : ', Kkma.nouns(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.nouns(s[:20]))
print('Komoran 형태소 분석 : ', Komoran.nouns(s[:20]))
print('Twitter 형태소 분석 : ', twitter.nouns(s[:20]))
print('Okt 형태소 분석 : ', okt.nouns(s[:20]))
# print('Mecab 형태소 분석 : ', mecab.nouns(sentence))
print('\n--------------------------------------------\n')

ex_sentence = '밀크T의 특징과 아이스크림 홈런의 특징 그리고 엘리하이의 특징에 대해서 알아보겠습니다.'
print('Kkma 형태소 분석 : ', Kkma.nouns(ex_sentence))
print('Hannaum 형태소 분석 : ', hannanum.nouns(ex_sentence))
print('Komoran 형태소 분석 : ', Komoran.nouns(ex_sentence))
print('Twitter 형태소 분석 : ', twitter.nouns(ex_sentence))
print('Okt 형태소 분석 : ', okt.nouns(ex_sentence))

print('\n-------------------------------------------\n')

from nltk import ngrams
from textblob import TextBlob

bigram = list(ngrams(ex_sentence.split(), 2))
print(bigram)
blob = TextBlob(ex_sentence)
blob.ngrams(n=2)

Kkma 형태소 분석 :  ['3', '3학년', '학년', '때', '밀크', '수업']
Hannaum 형태소 분석 :  ['3학년', '때', '밀크T', '수업', '시작']
Komoran 형태소 분석 :  ['학년', '때', '밀크', '수업', '시작']
Twitter 형태소 분석 :  ['학년', '때', '밀크', '수업', '시작']
Okt 형태소 분석 :  ['학년', '때', '밀크', '수업', '시작']

--------------------------------------------

Kkma 형태소 분석 :  ['밀크', '특징', '아이스', '아이스크림', '크림', '홈런', '엘', '엘리하이', '리', '하이']
Hannaum 형태소 분석 :  ['밀크T', '특징', '아이스크림', '홈런', '특징', '엘리하이', '특징']
Komoran 형태소 분석 :  ['밀크', '특징', '아이스크림', '홈런', '특징', '엘리', '하이', '특징']
Twitter 형태소 분석 :  ['밀크', '의', '특징', '아이스크림', '홈런', '특징', '엘리', '이의', '특징', '대해']
Okt 형태소 분석 :  ['밀크', '의', '특징', '아이스크림', '홈런', '특징', '엘리', '이의', '특징', '대해']

-------------------------------------------

[('밀크T의', '특징과'), ('특징과', '아이스크림'), ('아이스크림', '홈런의'), ('홈런의', '특징'), ('특징', '그리고'), ('그리고', '엘리하이의'), ('엘리하이의', '특징에'), ('특징에', '대해서'), ('대해서', '알아보겠습니다.')]


[WordList(['밀크T의', '특징과']),
 WordList(['특징과', '아이스크림']),
 WordList(['아이스크림', '홈런의']),
 WordList(['홈런의', '특징']),
 WordList(['특징', '그리고']),
 WordList(['그리고', '엘리하이의']),
 WordList(['엘리하이의', '특징에']),
 WordList(['특징에', '대해서']),
 WordList(['대해서', '알아보겠습니다'])]

### 단어 빈출 분석

#### 방법
1. 명사만 추출하여 빈출어 정리
    - 불용어 리스트를 만든 뒤, 불용어를 제외하여 형태소 분석 진행
    - 단어 빈도 추출
    - 단어들만 : 밀크T, 아이스크림, 태블릿, 
    - 단어 빈도 시각화
        - barchar
        - treemap
        - wordcloud
        <br>
1. 형태소 분석을 통해 각 문장에서 명사 / 형용사 등을 추출하여서 긍정적/부정적 키워드 분류 및 통계분석 (ex. 밀크T : 재밌게 하네요)

### 우선 Hannanum 으로 진행

### Hannanum
- 태그
    - N : 명사
        - NC : 보통명사
        - NQ : 고유명사
        - NB : 의존명사
        - NP : 대명사
        - NN : 수사
    
    - P : 용언
        - PV : 동사
        - PA : 형용사
        - PX : 보조용언

    - M : 수식언
        - MM : 관형사
        - MA : 부사
    
    - I : 독립언
    - J : 관계언
        - JC : 격조사
        - JX : 보조사
        - JP : 서술격조사

    - E : 어미
    - X : 접사

In [111]:
print('Hannaum 형태소 분석 : ', hannanum.nouns(s[:20]))
print('Hannaum 형태소 분석 : ', hannanum.pos(s[:20]))
a = hannanum.nouns(s[:20])
b = hannanum.pos(s[:20])

Hannaum 형태소 분석 :  ['3학년', '때', '밀크T', '수업', '시작']
Hannaum 형태소 분석 :  [('3학년', 'N'), ('때', 'N'), ('부터', 'J'), ('밀크T', 'N'), ('수업', 'N'), ('을', 'J'), ('시작', 'N'), ('하', 'X'), ('었고', 'E')]


In [112]:
print(b)
print(b[0][0], b[0][1])

[('3학년', 'N'), ('때', 'N'), ('부터', 'J'), ('밀크T', 'N'), ('수업', 'N'), ('을', 'J'), ('시작', 'N'), ('하', 'X'), ('었고', 'E')]
3학년 N


In [113]:
ex_sentence2 = '밀크T의 장점은 편하고 아이스크림 홈런의 장점은 귀엽고 엘리하이의 단점은 어려운 것 같아요'

In [114]:
from collections import Counter

test_noun = hannanum.nouns(df.contents[0])
test_count = Counter(test_noun)
test_top_nouns = dict(test_count.most_common(100))
test_top_nouns

{'학습': 6,
 '배터리': 6,
 '교체': 5,
 '수업': 4,
 '1년': 4,
 '밀크T': 3,
 '생각': 3,
 '충전': 3,
 '서비스센터': 3,
 '비용': 3,
 '재고': 3,
 '예약': 3,
 '시작': 2,
 '수학': 2,
 '학교': 2,
 '가능': 2,
 '중': 2,
 'A6': 2,
 '커버': 2,
 '막내': 2,
 '66,000원': 2,
 '30분': 2,
 '정도': 2,
 '그래서패드': 2,
 '3학년': 1,
 '때': 1,
 '2년': 1,
 '약정': 1,
 '시작해서': 1,
 '연장': 1,
 '연장해서': 1,
 '상태': 1,
 '처음': 1,
 '과목': 1,
 '코로나': 1,
 '기간': 1,
 '있어서잘했다': 1,
 '사회': 1,
 '과학': 1,
 '관련': 1,
 '내용을수': 1,
 '코딩': 1,
 '자': 1,
 '중국어': 1,
 '학원필요': 1,
 '1주일': 1,
 '12회': 1,
 '초등': 1,
 '2020년': 1,
 '시작할갤럭시탭': 1,
 '기계': 1,
 '받았다지금은주는지': 1,
 '밀크아이': 1,
 '갤럭시탭S': 1,
 '시리즈': 1,
 '보는같다패드': 1,
 '힘들어서번도하다가이번': 1,
 '일': 1,
 '진짜': 1,
 '새것': 1,
 '같음밀크T': 1,
 '화면': 1,
 '일반': 1,
 '탭': 1,
 '사용': 1,
 '가능하지만번': 1,
 'ㅎㅎ그래서': 1,
 '우리남매': 1,
 '한다3년하': 1,
 '1개월': 1,
 '지난': 1,
 '여태사용했는데,': 1,
 '주말': 1,
 '지': 1,
 '3시간': 1,
 '1에서이상': 1,
 '전원': 1,
 '0': 1,
 '1회': 1,
 '전화맞춤': 1,
 '코칭': 1,
 '화상수업': 1,
 '날': 1,
 '어려울같아서': 1,
 '안될같다': 1,
 '담당': 1,
 '선생님': 1,
 '낮잠': 1,
 '월요일': 1,
 '붐벼서갔다갤럭시탭': 1

In [115]:
test_dict = {}
test_dict['a'] = ['a']
print(test_dict.get('a'))

test_dict.get('a').append('b')
print(test_dict.get('a'))

['a']
['a', 'b']


In [116]:
icecream_list = []
elihigh_list = []

for i in df['contents']:
    if '아이스크림' in i:
        icecream_list.append(True)

    else:
        icecream_list.append(False)

for i in df['contents']:
    if '엘리하' in i:
        elihigh_list.append(True)

    else:
        elihigh_list.append(False)

In [117]:
def keyword_search(df, key1):

    key1_list = []
    key2_list = []

    for i in df['contents']:
        if '아이스크림' in i:
            key1_list.append(True)

        else:
            key1_list.append(False)

    df[key1] = key1_list
    return df

In [118]:
df_2 = df.copy()
df_2.drop('contents_list', axis=1, inplace=True)
df_2
# keyword_search(df, '아이스크림')

,contents,recommend
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함. 연장해서 1년...",False
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바로...",True
2,"밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요2...",True
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...,True
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요영어 수학...,False
...,...,...
118,새로운 선생님과 친구들을 만날 생각에 신이 나서 개학을 기다렸는데학교도 개학이 연기...,False
119,이제 3학년이 되는 아드님은 오늘도 열심히 밀크T학습을 하면서 학습을 진행하고 있어...,False
120,초등인강 추천아이 눈높이에 딱요 며칠간 아이들이 정말 신나 보였어요. 이렇게 신나있...,False
121,안녕하세요. 강북구과외 선생님입니다. 학생들을 지도한지 5년 정도 되었습니다. 강북...,False


In [119]:
df_contents = df.copy()
df_contents.drop('contents_list', axis=1, inplace=True)
df_contents['icecream'] = icecream_list
df_contents['elihigh'] = elihigh_list
df_contents

,contents,recommend,icecream,elihigh
0,"3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함. 연장해서 1년...",False,False,False
1,"밀크티패드를 시작한지 2개월을 채워가는 중이네요 작년 12월 말, 급한 마음에 바로...",True,False,False
2,"밀크T 친구 추천 이벤트밀크티 학습지 남매 2년 사용, 연장 후기3월9일수정 해요2...",True,True,False
3,새학기 되면서 아이들 학원 스케줄 짜느라 분주한 분들 많으시죠. 저는 올해 중학생이...,True,False,False
4,연두는 현재 6학년으로 예비중등입니다. 저학년 때부터 밀크티에 정착했는데요영어 수학...,False,False,False
...,...,...,...,...
118,새로운 선생님과 친구들을 만날 생각에 신이 나서 개학을 기다렸는데학교도 개학이 연기...,False,True,False
119,이제 3학년이 되는 아드님은 오늘도 열심히 밀크T학습을 하면서 학습을 진행하고 있어...,False,False,False
120,초등인강 추천아이 눈높이에 딱요 며칠간 아이들이 정말 신나 보였어요. 이렇게 신나있...,False,False,False
121,안녕하세요. 강북구과외 선생님입니다. 학생들을 지도한지 5년 정도 되었습니다. 강북...,False,False,False


In [120]:
df_test = df_contents[df_contents['elihigh'] == True]
df_test = df_test[df_test['icecream'] == True]
df_test.reset_index(drop=True, inplace=True)
df_test

,contents,recommend,icecream,elihigh
0,코로나 시국 초기에 시작한 밀크T가 벌써 2년이 되어서 연장을 하게 되었다. 코로나...,True,True,True
1,이제 초등학교 고학년인 아들. 매일 뒹굴거리며 심심하다 노래 부른다. 학원은 태권도...,True,True,True
2,"초등 인강 밀크T, 엘리하이, 와이즈캠프 체험올해 6학년이 되는 우리 큰아이는 영어...",False,True,True
3,초등학생 인강홈런 VS 밀크티 온라인학습 비교 무료체험 솔직 후기 홈런 오른 밀크티...,True,True,True
4,안녕하세요해피블루입니다. 즐거운 주말 보내고 계신가요주말은 밍기적거리면서 낮잠도 자...,True,True,True


In [121]:
sentence_96 = df_test['contents'][3]
sentence_96[1000:2000]

'죄송합니다 띠로리 밀크티 오른 홈런그런 저도 어쩔없었습니다 ㅋㅋ정말 이건 시킬 생각 없다며 자신했었는데코로오나 네가발목을 잡았구나. 아놔 ㅠㅠ이왕이면아이에게 지면수업 위주의 교육을 시키고싶었고, 인터넷강의 패드수업 온라인강의 등은최대한 늦게 늦게 접하게주려고했었는데. 무너짐요1년 동안 아무 교육도시키고, 학원도중단하고 나니. 정말공허함과 불안감은이루 말할없었습니다즤 집은 애미 혼자 문제 신랑이 중간중간 제가 고민할 때마다괜찮다, 학원 대신 우리 승준이는다른 것을얻은 것이 많다, 걱정하지 말자. 등등의다양한 위로를 해주며 함께 교육했지만. 이놈의 ㅈ1럴 맞은 애어매 성격상. 신랑처럼 평화로운 마음의 소유자가 아니므로저는 아이가 초등학교 3학년이 되는 올해1월부터 당장 시작해야겠다고마음 먹었습니다 무엇바로왼 밀크티 오른 홈런 펜을 꺼내어 수업해요 초등인강 초등인터넷강의초등패드수업 초등홈스쿨 대명사인밀크T VS 홈런 이요. 즈이 신랑은 무료체험으로 밀크티수업 홈런수업 2가지신청했다는말을 듣고는처음에는 살짜콩 놀라는 눈치였지만너무했다고, 좋은 점이 분명 있을 테니 학원대신 현재 집에서 우리가 시키고 있는 홈스쿨과 같이 추가로 시너지 효과 내기에좋은 프로그램이거라고절 칭찬해 주더라고요. 푸하하 ㅋㅋㅋ뭐든 제가 저지른 일들을 칭찬하고보는 즈이 신랑입니다 아놔 ㅋㅋㅋ 공감해주니 고마운 단짝이지요. 아이 학습도 언제나 함께해 주니 제가 그나마 스트레스의 늪에서 잘빠져 나오고 있고요. 엄마표수업아빠표수업 공존해야합니다 그렇다면각각 10일 동안의 무료체험 수업을 해보고 나서 느낀 솔직 후기를 전해 드릴게요. 다만, 저희 아이와 저희 부부의 의견이기 때문에 참고만주세요2가지 온라인학습지아이스크림홈런먼저 소개할게요. 결론 먼저 말씀 드리면 2가지중 정말 고민이수밖에 없는 것이둘정말 좋아요. ㅠㅠ그래서꼬오오오오옥아이에게 무료체험학습 시켜 보신 후결정하시길 아주아주 격하게 추천드리는 바입니다. 아이에 따라 선호도가 정말제각각이라 체험수업 필수예요. 일단홈런 태블릿 학습기 스타일 자체

### 설계해보자...

key1 = '밀크'
key2 = '홈런'

1. key값이 나올때마다 문장을 끊어준다
    - key값이 목적어처럼 쓰이는 경우는...? -> 형태소분석 파트에서 주어를 분석해주는 기능을 사용해야 할 듯 하다.
    - solution1
        - key1 값이 나올때마다 split을 사용하여 문장을 끊어준다.
        - if문을 사용하여 split된 문장 중에 key2 값이 있다면, 따로 리스트에 추가한다.
    
2. 나눠진 문장을 리스트에 넣어준다 (split 사용시 skip)
3. 해당 리스트를 반복문을 사용하여 형태소 분석을 실시한다.
    - kkma를 사용하여 동사/형용사를 추출한다.
        - 속도가 느리다면 hannanum으로 실행하고 P 항목에만 kkma를 사용한다.
1. key값 별로 출현한 동사/형용사를 구분지어준다.
1. 단어별 빈도를 추출한다.

In [122]:
# 주어 찾기 함수
def subject_extraction(text):
    # 형태소 분석
    pos = Kkma.pos(text)

    # 주어 추출
    for i in range(len(pos)):
        if pos[i][1] == 'NNG':  # 명사
            if i == 0 or (i > 0 and pos[i-1][1] != 'NNG'):  # 주어는 문장 첫 단어이거나 앞 단어가 조사가 아닌 경우
                return pos[i][0]
                break

In [123]:
# sentence_list = sentence_96.split('.')
# sentence_milk = []
# sentence_homerun = []
# sentence_dummy = []

# for i in range(len(sentence_list)):
#     if subject_extraction(sentence_list[i]) == '밀크':
#         plus_num = 1
#         milk_para = []
#         milk_para.append(sentence_list[i])
#         if subject_extraction(sentence_list[i+plus_num]) == '밀크':
#             pass
#         else:
#             milk_para.append(sentence_list[i+plus_num])
#             plus_num += 1
#         sentence_milk.append(sentence)

#     elif subject_extraction(sentence) == '홈런':
#         sentence_homerun.append(sentence)

#     else:
#         sentence_dummy.append(sentence)

In [124]:
# 키워드가 주어인 본문 정리하기
## keword에서 다음 keyword 까지의 text 추출


def gather_sentence(text, key1, key2):

    text_list = text.split('.') # 문장 단위 : "."

    while ' ' in text_list: # '' 안에 text가 들어있지 않으면 error가 발생함을 방지
        text_list.remove(' ')

    key1_paragraph = []
    key2_paragraph = []
    for i in range(len(text_list)):
        if subject_extraction(text_list[i]) == key1:
            plus_num = 1
            # milk_para.append(text_list[i])
            sentence = text_list[i]
            # print(f'{key1}파트 {i}번째 진행중')
            while i+plus_num < len(text_list):
                if subject_extraction(text_list[i+plus_num]) == key1 or subject_extraction(text_list[i+plus_num]) == key2:
                    # print(f'!! 스캔 {key1}파트 종료')
                    break
                
                else:
                    # milk_para.append(text_list[i+plus_num])
                    if i+plus_num > len(text_list):
                        # print(f'!! 스캔 {key1}파트 종료')
                        break
                    else:
                        sentence = sentence + '' + text_list[i+plus_num]
                        plus_num += 1
                        # print(f'!! 스캔 {key1}파트 {i+plus_num}번째 진행중')

            key1_paragraph.append(sentence)

        else:
            pass

        if subject_extraction(text_list[i]) == key2:
            plus_num = 1
            # milk_para.append(text_list[i])
            sentence = text_list[i]
            # print(f'{key2}파트 {i}번째 진행중')
            while i+plus_num < len(text_list):
                if subject_extraction(text_list[i+plus_num]) == key1 or subject_extraction(text_list[i+plus_num]) == key2:
                    break
                
                else:
                    # milk_para.append(text_list[i+plus_num])
                    sentence = sentence + '' + text_list[i+plus_num]
                    plus_num += 1

            key2_paragraph.append(sentence)

        else:
            pass

    if key1_paragraph == key2_paragraph:
        for i in range(len(text_list)):
            pos_list = Kkma.pos(text_list[i])
            if (key1, 'NNG') in pos_list:
                key1_paragraph.append(text_list[i])

            if (key2, 'NNG') in pos_list:
                key2_paragraph.append(text_list[i])
    else:
        pass
    
    return {key1 : key1_paragraph, key2 : key2_paragraph}

In [125]:
# def gather_sentence(text, key):

#     text_list = text.split('.') # 문장 단위 : "."
#     key_paragraph = []
#     for i in range(len(text_list)):
#         if subject_extraction(text_list[i]) == key:
#             plus_num = 1
#             # key_paragraph.append(text_list[i])
#             sentence = text_list[i]
#             while i+plus_num < len(text_list):
#                 if subject_extraction(text_list[i+plus_num]) == key:
#                     break

#                 else:
#                     # key_paragraph.append(text_list[i+plus_num])
#                     sentence = sentence + ' ' + text_list[i+plus_num]
#                     plus_num += 1

#             key_paragraph.append(sentence)

#         else:
#             pass

#     dict = {key : key_paragraph}

#     return dict

In [126]:
subject_extraction('밀크티 태블릿이 홈런 태블릿보다 좋다')

'밀크'

In [127]:
text_list = ['밀크티마이쪄', '응애', '밀크티배고파', '밀크태블릿', '밀농사', '어흥', '밀크티 태블릿이 홈런 태블릿보다 좋다', '우와', '홈런을 칩니다', '최고야', '밀크티학습']
text_test = ''

for i in text_list:
    text_test = text_test + '. ' + i

text_test

'. 밀크티마이쪄. 응애. 밀크티배고파. 밀크태블릿. 밀농사. 어흥. 밀크티 태블릿이 홈런 태블릿보다 좋다. 우와. 홈런을 칩니다. 최고야. 밀크티학습'

In [128]:
gather_sentence(text_test, '밀크', '홈런')

{'밀크': [' 밀크티마이쪄 응애',
  ' 밀크티배고파',
  ' 밀크태블릿 밀농사 어흥',
  ' 밀크티 태블릿이 홈런 태블릿보다 좋다 우와',
  ' 밀크티학습'],
 '홈런': [' 홈런을 칩니다 최고야']}

In [129]:
coffee = '밀크T 수업을 시작하고 1년이 지났다. 연장은 2년을 하였으며 처음엔 수학과목만 생각해서 시작을 했는데 홈런보다 영어 컨텐츠가 더 좋은 것 같아서 연장했다. 학교가기 전에 밀크티로 예습하면 수업이 재미있다고 한다. 아이들 말로는 홈런은 캐릭터가 재미있다고 한다. 홈런이 캐릭터 하나는 기가막히게 만드나보다. 혹시라도 홈런과 밀크티를 고민하는 분이 있으면 이 글을 참고해보세요.'
gather_sentence(coffee, '밀크', '홈런')

{'밀크': ['밀크T 수업을 시작하고 1년이 지났다 연장은 2년을 하였으며 처음엔 수학과목만 생각해서 시작을 했는데 홈런보다 영어 컨텐츠가 더 좋은 것 같아서 연장했다 학교가기 전에 밀크티로 예습하면 수업이 재미있다고 한다 아이들 말로는 홈런은 캐릭터가 재미있다고 한다'],
 '홈런': [' 홈런이 캐릭터 하나는 기가막히게 만드나보다', ' 혹시라도 홈런과 밀크티를 고민하는 분이 있으면 이 글을 참고해보세요']}

In [130]:
milk_t.contents[0]

'3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서, 또 연장함.\n연장해서 1년이 지났고 1년이 남은 상태.\n처음엔 수학 과목만 생각해서 시작을 했는데 코로나로 인해 학교 못 가는 기간이 있어서 참 잘했다 싶다.\n사회, 과학 관련 내용을 볼 수 있다.\n코딩도 하고, 한자, 중국어도 가능하니.... 따로 학원 갈 필요 없이 1주일에 1-2회 보며 학습하는 중.\n밀크T 초등 2020년에 시작할 때 갤럭시탭 A6를 기계로 받았다!\n지금은 뭘 주는지 모름;;;; 밀크 T 아이는 갤럭시탭S 시리즈는 받았다고 들음.\n처음 커버 벗겨 보는 것 같다.\n키 패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서 한 번도 안 하다가\n이번에 AS 받을 일이 있어서 벗겨본다.\n진짜 새것 같음!\n밀크T 학습 화면으로 되어있는데, 나가기를 누르면... 일반 탭으로도 사용이 가능하지만,\n한 번도 나가기를 누르지 않음. ㅎㅎ\n그래서 우리 집 남매는 모르는.................... 계속 몰랐으면 한다!\n3년하고 1개월이 지난 것 같다.\n여태 잘 사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1%에서 더 이상 올라가지 않는다. 전원을 켜면 배터리는 다시 0%로 꺼짐.\n학습을 하고 매주 1회 전화 또 맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다.\n충전이 어려울 것 같아서 학습도 안되고 수업도 안될 것 같다.\n담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,\n막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서 안 갔다!\n갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다. (1588-3366)\n배터리 교체 비용을 물으니 66,000원이라고 한다.\n더불어 어느 센터로 갈 건지 알려주시면 재고 확인을 해주신다고 한다.\n집 근처 가까운 곳을 알려드렸다.\n배터리 재고는 딱 하나 있다고 하셨다.\n재고 예약도 가능하다고 한다.\n와~~~ 그냥 갔는데 없으면 두 번

In [131]:
df.contents[0]

'3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함. 연장해서 1년이 지났고 1년이 남은 상태. 처음엔 수학 과목만 생각해서 시작을 했는데 코로나로 인해 학교가는 기간이 있어서잘했다 싶다. 사회, 과학 관련 내용을수 있다. 코딩도 하고, 한자, 중국어도 가능하니. 따로 학원필요 없이 1주일에 12회 보며 학습하는 중. 밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음. 처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다. 진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면. 일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음. ㅎㅎ그래서 우리남매는 모르는. 계속 몰랐으면 한다3년하고 1개월이 지난같다. 여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다. 전원을 켜면 배터리는 다시 0로 꺼짐. 학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다. 충전이 어려울같아서 학습도 안되고 수업도 안될같다. 담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다. 15883366배터리 교체 비용을 물으니 66,000원이라고 한다. 더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다. 배터리 재고는하나 있다고 하셨다. 재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다. 하나 남은 재고 예약까지 했다. 2주 이내로 방문하면 된다고 함. 예약하고 다음날 바로 감. 배터리 교체하고 학습해야 하니까. 오후에 가니 확실히붐볐다. 대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다. 배터리 교체

In [132]:
test_def = df.contents[0]
display('홈런' in test_def)
display(test_def)

False

'3학년 때부터 밀크T 수업을 시작했고 2년 약정으로 시작해서,연장함. 연장해서 1년이 지났고 1년이 남은 상태. 처음엔 수학 과목만 생각해서 시작을 했는데 코로나로 인해 학교가는 기간이 있어서잘했다 싶다. 사회, 과학 관련 내용을수 있다. 코딩도 하고, 한자, 중국어도 가능하니. 따로 학원필요 없이 1주일에 12회 보며 학습하는 중. 밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음. 처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다. 진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면. 일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음. ㅎㅎ그래서 우리남매는 모르는. 계속 몰랐으면 한다3년하고 1개월이 지난같다. 여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다. 전원을 켜면 배터리는 다시 0로 꺼짐. 학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다. 충전이 어려울같아서 학습도 안되고 수업도 안될같다. 담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다. 15883366배터리 교체 비용을 물으니 66,000원이라고 한다. 더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다. 배터리 재고는하나 있다고 하셨다. 재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다. 하나 남은 재고 예약까지 했다. 2주 이내로 방문하면 된다고 함. 예약하고 다음날 바로 감. 배터리 교체하고 학습해야 하니까. 오후에 가니 확실히붐볐다. 대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다. 배터리 교체

In [133]:
gather_sentence(test_def, '밀크', '홈런')

{'밀크': [' 밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음 처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다 진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면 일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음 ㅎㅎ그래서 우리남매는 모르는 계속 몰랐으면 한다3년하고 1개월이 지난같다 여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다 전원을 켜면 배터리는 다시 0로 꺼짐 학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다 충전이 어려울같아서 학습도 안되고 수업도 안될같다 담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다 15883366배터리 교체 비용을 물으니 66,000원이라고 한다 더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다 배터리 재고는하나 있다고 하셨다 재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다 하나 남은 재고 예약까지 했다 2주 이내로 방문하면 된다고 함 예약하고 다음날 바로 감 배터리 교체하고 학습해야 하니까 오후에 가니 확실히붐볐다 대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다 배터리 교체 비용 66,000원 결제 완료 사실, 1년 남은 시점이고 내년엔 중등으로 넘어가야 한다 아직까진 중학생이 되어도 밀크T를 계속할 생각이다 그래서패드로 교체를생각도 했는데6학년이지금 학교 마치고 방과후수업과 영어, 수학 학원을 다녀오면 7시시간이 있을까라는 생각이 든다 아직 6학년의 생활을 시작도했고봐야것 같아서남은 1년 동안 계속 유지가 될지 중간에 그만

In [134]:
# test_split = sentence_96.split(' ')
# num = 0
# word = input('단어')
# for i in test_split:
#     if word in i:
#         num += 1

# print(word, ':',num)

In [135]:
split_milk = sentence_96.split('밀크')
list_milk = []
list_homerun = []

for i in split_milk:
    if '홈런' in i:
        list_homerun.append(i)
    else:
        list_milk.append(i)
        # list_milk.append(Kkma.pos(i))

In [136]:
# text = "밀크티 무료체험 해보니 쉽고 좋아요 이제 해지하려구요"
text = "밀크티 무료체험 해보니 아이가 게임을 하며 즐거워하는데 너무 게임만 할까봐 걱정이네요."

pos = Kkma.pos(text)

# 주어 추출
for i in range(len(pos)):
    if pos[i][1] == 'NNG':  # 명사
        if i == 0 or (i > 0 and pos[i-1][1] != 'NNG'):  # 주어는 문장 첫 단어이거나 앞 단어가 조사가 아닌 경우
            print(pos[i][0])
            break

밀크


In [137]:
Kkma.tagset
# VA : 형용사
# VV : 동사 - 얜 추출 안해도 될듯...?
# NNG : 보통명사
# VA, NNG만 추출

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [138]:
pos

[('밀크', 'NNG'),
 ('티', 'NNG'),
 ('무료', 'NNG'),
 ('체험', 'NNG'),
 ('해보', 'VV'),
 ('니', 'ECD'),
 ('아이', 'NNG'),
 ('가', 'JKS'),
 ('게임', 'NNG'),
 ('을', 'JKO'),
 ('하', 'VV'),
 ('며', 'ECE'),
 ('즐거워하', 'VV'),
 ('는데', 'ECD'),
 ('너무', 'MAG'),
 ('게임', 'NNG'),
 ('만', 'JX'),
 ('하', 'VV'),
 ('ㄹ까', 'ECD'),
 ('보', 'VV'),
 ('아', 'ECS'),
 ('걱정', 'NNG'),
 ('이', 'VCP'),
 ('네요', 'EFN'),
 ('.', 'SF')]

In [139]:
hannanum.pos(text)

[('밀크티', 'N'),
 ('무료체험', 'N'),
 ('하', 'P'),
 ('어', 'E'),
 ('보', 'P'),
 ('니', 'E'),
 ('아이', 'N'),
 ('가', 'J'),
 ('게임', 'N'),
 ('을', 'J'),
 ('하', 'P'),
 ('며', 'E'),
 ('즐겁', 'P'),
 ('어', 'E'),
 ('하', 'P'),
 ('ㄴ데', 'E'),
 ('너무', 'M'),
 ('게임', 'N'),
 ('만', 'J'),
 ('할까봐', 'N'),
 ('걱정', 'N'),
 ('이', 'J'),
 ('네', 'E'),
 ('요', 'J'),
 ('.', 'S')]

In [140]:
# keyword_list = ['밀크T', '아이스크림', '엘리하이']
# hann_list = {}

# for i in hannanum.pos(ex_sentence2):
#     if i[1] == 'N':
#         hann_list.append(i[0])

#     elif i[1] == 'P':
#         hann_list.append(i[0])

    

# print('N : ', N_list)
# print('P : ', P_list)

In [141]:
text

'밀크티 무료체험 해보니 아이가 게임을 하며 즐거워하는데 너무 게임만 할까봐 걱정이네요.'

In [142]:
Kkma.nouns(text)

['밀크', '밀크티', '티', '무료', '무료체험', '체험', '아이', '게임', '걱정']

In [143]:
A = []

for i in hannanum.nouns(text):
    A.append(Kkma.pos(i))

In [144]:
A

[[('밀크', 'NNG'), ('티', 'NNG')],
 [('무료', 'NNG'), ('체험', 'NNG')],
 [('아이', 'NNG')],
 [('게임', 'NNG')],
 [('게임', 'NNG')],
 [('하', 'VV'), ('ㄹ까', 'ECD'), ('보', 'VV'), ('아', 'ECS')],
 [('걱정', 'NNG')]]

In [145]:
Kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

In [146]:
test = ' 밀크T 초등 2020년에 시작할갤럭시탭 A6를 기계로 받았다지금은주는지 모름 밀크아이는 갤럭시탭S 시리즈는 받았다고 들음 처음 커버 벗겨 보는같다패드가 달린 파란 커버가 씌워져있는데 벗기기 힘들어서번도하다가이번에 AS 받을 일이 있어서 벗겨본다 진짜 새것 같음밀크T 학습 화면으로 되어있는데, 나가기를 누르면 일반 탭으로도 사용이 가능하지만번도 나가기를 누르지 않음 ㅎㅎ그래서 우리남매는 모르는 계속 몰랐으면 한다3년하고 1개월이 지난같다 여태사용했는데, 주말이 지나고 충전하려고 하는데 3시간이 넘게 충전을 해도 1에서이상 올라가지 않는다 전원을 켜면 배터리는 다시 0로 꺼짐 학습을 하고 매주 1회 전화맞춤 코칭으로 화상수업을 하는데 수업이 있는 날이다 충전이 어려울같아서 학습도 안되고 수업도 안될같다 담당 선생님께 말씀드려서 수업을 미웠고 바로 서비스센터에 가고 싶었지만,막내는 낮잠을 자고 있고 월요일은 서비스센터가 붐벼서갔다갤럭시탭 A6 배터리 교체 비용이 궁금해서 삼성전자 고객센터로 전화를 했다 15883366배터리 교체 비용을 물으니 66,000원이라고 한다 더불어 어느 센터로건지 알려주시면 재고 확인을 해주신다고 한다근처 가까운 곳을 알려드렸다 배터리 재고는하나 있다고 하셨다 재고 예약도 가능하다고 한다그냥 갔는데 없으면번 가야뻔했다 하나 남은 재고 예약까지 했다 2주 이내로 방문하면 된다고 함 예약하고 다음날 바로 감 배터리 교체하고 학습해야 하니까 오후에 가니 확실히붐볐다 대기 없이 바로 엔지니어 분께서 불렀고, 30분 정도 걸린다고 하셨으나15분 정도 있으니 바로주셨다 배터리 교체 비용 66,000원 결제 완료 사실, 1년 남은 시점이고 내년엔 중등으로 넘어가야 한다 아직까진 중학생이 되어도 밀크T를 계속할 생각이다 그래서패드로 교체를생각도 했는데6학년이지금 학교 마치고 방과후수업과 영어, 수학 학원을 다녀오면 7시시간이 있을까라는 생각이 든다 아직 6학년의 생활을 시작도했고봐야것 같아서남은 1년 동안 계속 유지가 될지 중간에 그만둘지 아무도 모른다 그래서패드로 교환보다는 배터리 교체를 했다 5학년 초반까지만 해도 스스로 자기주도학습이되었는데 점점 하라고 하라고 해야 하는 중 그래서 얼마나할있을지 모르겠다 그렇지만, 학습하는 내용은 아주 도움이 되니까, 계속 유지하길 바라는마음 서비스센터가 집에서 가까워서 막내 유모차 태우고 걸어갔는데,왕복 30분 거리를 걸어갔다 왔더니캐시워크가 오늘 하루 만보를 걸었단다 괜히 뿌듯'

In [147]:
emoji_test = '히히😅땀땀'

emoji_result = core.replace_emoji(emoji_test, replace='')
emoji_result

'히히땀땀'

In [148]:
test_kk = Kkma.pos(test)

In [149]:
test_kk[0][1]

'NNG'

In [150]:
df.contents[118]

'새로운 선생님과 친구들을 만날 생각에 신이 나서 개학을 기다렸는데학교도 개학이 연기되고 학원도 휴원이 연장되어 집에만 있네요아들이 좋아하는 게임도 하면서 늘어난 방학을 신나게 즐기고 있어요문제집을 풀라고 하면 대답만 하고 한쪽으로 밀어두던 아이가 밀크티는 매일 학습하니 얼마나 다행인지 몰라요그래서 초등수학학습지 추천 해달라고 하면 밀크티 초등을 많이 이야기 하나 봐요6학년이 되니 수학이어려워져서 힘들어한다는데문제만 푸는 문제집과 달리 영상으로 개념 이해를 도와주고문제도더 쉽게 자세히 풀이 과정을 설명해 주니아직까지 아들은 수학이 그리 어렵게 느껴지지 않는같아요밀크티는 초등수학학습지 추천외에도 다른 과목도 체계적으로 학습을수 있다는 점이 좋아요독서를 좋아하는 아이가 아니라서 독서논술이 걱정이었는데6학년 과목 학습을 하면서 독서논술도 함께수 있어서 너무 좋더라고요방학 동안에도 책은권 읽지를 않았는데그래도 밀크티 독서논술에 나오는 책들은 읽더라고요우린 EBOOK이 없던 시대라 종이책만 보아서인지 종이책이 익숙한데요즘 아이들에게는 EBOOK이익숙한같아요아이들의 상상의 나래를 펼쳐주는 흥미로운 이야기라 그런지피터팬은재미있게 읽네요어휘력도 재미있게 쌓아요재생 수200000 0038화질 선택 옵션자동어휘력도 재미있게 쌓아요피터팬을 읽고 나서 낱말의 뜻을 듣고뜻에 맞는 낱말을 찾아보는 건데뜻을 듣고 해당하는 낱말을 암기하는아니라이렇게 게임처럼 배우니 딱딱하고 어렵게 느껴질있는 어휘도 한결 쉽게 배울있어요남자아이들이 대부분 과학을 좋아하죠하지만 과학도 학년이 올라갈수록점점 난이도가 높아지니 저학년 때처럼 재미있지만은 않아요문제 인식과 가설 설정이란 말만 보면 어려울같지만선생님께서 하나하나 차근차근 설명해 주시니설명을 듣고 나면 어렵지 않답니다부분을 만약에 학습지로 혼자서 공부했다면 쉽게 이해하기가 어려웠을같아요생활실험을 보면 재미도 있지만재미 속에 과학의 원리도 담겨 있답니다. 드라이아이스 비눗방울 만들기를 보더니 자기도 해보고 싶다는데당장 집에 드라이아이스가 없어서 아이스

In [151]:
test = Kkma.pos(df.contents[119].split('.')[0])
test

[('이제', 'NNG'),
 ('3', 'NR'),
 ('학년', 'NNG'),
 ('이', 'JKC'),
 ('되', 'VV'),
 ('는', 'ETD'),
 ('아드님', 'NNG'),
 ('은', 'JX'),
 ('오늘', 'NNG'),
 ('도', 'JX'),
 ('열심히', 'MAG'),
 ('밀크', 'NNG'),
 ('T', 'OL'),
 ('학습', 'NNG'),
 ('을', 'JKO'),
 ('하면서', 'NNG'),
 ('학습', 'NNG'),
 ('을', 'JKO'),
 ('진행하', 'VV'),
 ('고', 'ECE'),
 ('있', 'VV'),
 ('어요', 'EFN')]

In [152]:
hannanum.pos(df.contents[119].split('.')[0])

[('이제', 'M'),
 ('3학년', 'N'),
 ('이', 'J'),
 ('되', 'P'),
 ('는', 'E'),
 ('아드님', 'N'),
 ('은', 'J'),
 ('오늘', 'N'),
 ('도', 'J'),
 ('열심히', 'M'),
 ('밀크T학습', 'N'),
 ('을', 'J'),
 ('하', 'P'),
 ('면서', 'E'),
 ('학습', 'N'),
 ('을', 'J'),
 ('진행', 'N'),
 ('하고', 'J'),
 ('있', 'P'),
 ('어요', 'E')]

In [153]:
('밀크', 'NNG') in test

True

In [154]:
# key1 = '홈런'
# key2 = input('find keyword')
# 이모티콘 같은 특수기호가 들어가 있으면 에러발생...

df['keyword_dict'] = df['contents'].apply(lambda x : gather_sentence(x, '밀크', '홈런'))

KeyboardInterrupt: 

In [155]:
dict_list = []

for i in range(len(df.contents)):
    print(f'{i}번째 실행중')
    dict_list.append(gather_sentence(df.contents[i], '밀크', '홈런'))

0번째 실행중
1번째 실행중
2번째 실행중
3번째 실행중
4번째 실행중
5번째 실행중
6번째 실행중
7번째 실행중
8번째 실행중
9번째 실행중
10번째 실행중
11번째 실행중
12번째 실행중
13번째 실행중
14번째 실행중
15번째 실행중
16번째 실행중
17번째 실행중
18번째 실행중
19번째 실행중
20번째 실행중
21번째 실행중
22번째 실행중
23번째 실행중
24번째 실행중
25번째 실행중
26번째 실행중
27번째 실행중
28번째 실행중
29번째 실행중
30번째 실행중
31번째 실행중
32번째 실행중
33번째 실행중
34번째 실행중
35번째 실행중
36번째 실행중
37번째 실행중
38번째 실행중
39번째 실행중
40번째 실행중
41번째 실행중
42번째 실행중
43번째 실행중
44번째 실행중
45번째 실행중
46번째 실행중
47번째 실행중
48번째 실행중
49번째 실행중
50번째 실행중
51번째 실행중
52번째 실행중
53번째 실행중
54번째 실행중
55번째 실행중
56번째 실행중
57번째 실행중
58번째 실행중
59번째 실행중
60번째 실행중
61번째 실행중
62번째 실행중
63번째 실행중
64번째 실행중
65번째 실행중
66번째 실행중
67번째 실행중
68번째 실행중
69번째 실행중
70번째 실행중
71번째 실행중
72번째 실행중
73번째 실행중
74번째 실행중
75번째 실행중
76번째 실행중
77번째 실행중
78번째 실행중
79번째 실행중
80번째 실행중
81번째 실행중
82번째 실행중
83번째 실행중
84번째 실행중
85번째 실행중
86번째 실행중
87번째 실행중
88번째 실행중
89번째 실행중
90번째 실행중
91번째 실행중
92번째 실행중
93번째 실행중
94번째 실행중
95번째 실행중
96번째 실행중
97번째 실행중
98번째 실행중
99번째 실행중
100번째 실행중
101번째 실행중
102번째 실행중
103번째 실행중
104번째 실행중
105번째 실행중
106번째 실행중
107번째 실행중
108번째 실행중
109번째 실행중
110번째 실행중


In [ ]:
import pandas as pd

In [ ]:
ele_df = pd.read_parquet('./test_final.parquet')
ele_df

,text,date,click,topic
0,어떤게나을까요?! 지금은 웅진 학습지로만(지면) 하고있어요~ 패드로 슬슬 갈까하는데...,2023-03-03 14:49:00,502,초1
1,어떤게나을까요?! 지금은 웅진 학습지로만(지면) 하고있어요~ 패드로 슬슬 갈까하는데...,2023-03-03 14:49:00,502,초1
2,초1이고 밀크티 하고 있어요\n오늘의 학습만 매일하고 있는데요\n초등 2학년때까진 ...,2023-03-20 22:35:00,208,초1
3,"대부분의 받아쓰기는 빠르면 4살, 늦어도\n초등학교 입학전부터 학습하는 경우가 많잖...",2022-09-06 16:58:00,661,초1
4,"초1인데 패드로 하는 학습지를 시켜보고 싶은데요.\n빨간펜, 윙크,밀크티 중 어떤게...",2023-03-14 15:02:00,219,초1
...,...,...,...,...
1792,아이가 구몬국어랑 수학을 3년째 하고 있어요. 잘따라와줬고 밀리지않고 잘하는 편인데...,2021-01-08 00:16:00,4056,초3
1793,초등1 여아이에요.\n서점에 들렸다가 밀크티 알게 되었는데요~\n학년별로 다 나왔있...,2020-02-10 10:40:00,773,초3
1794,밀크티나 홈런? 엘리하이 이런거는 어디서 체험신청하나요? 초5인데 관리잘되는곳은 어...,2021-12-06 00:27:00,1094,초3
1795,"학원은 안가려고 해서\n패드수업알아보는데요~\n현재 초3,초1이에요\n밀크티 와이즈...",2022-09-22 21:24:00,564,초3


In [ ]:
# df = pd.read_csv('./밀크T 아이스크림.csv')
# df.contents[0]